### Setup and Imports

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
import config
from src import data_ingestion
from src.symbols import Symbols
from src.portfolio1 import Portfolio
from src.benchmark import Benchmark

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Load Transaction Data

In [2]:
# Create a list of all your transaction log files
log_files = [
    config.TRANS_LOG_DIR / "us_mkt_transactions.csv",
    config.TRANS_LOG_DIR / "exus_mkt_transactions.csv",
]

# Load and merge all found logs into a single DataFrame
master_log = data_ingestion.create_master_log(log_files).set_index("Date")
display(master_log)

Successfully loaded log: us_mkt_transactions.csv
Successfully loaded log: exus_mkt_transactions.csv


,Type,Symbol,Quantity,Price,Amount,Commission,Currency,Description,Exchange,Source
Date,,,,,,,,,,
2023-02-16,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,ID: 37fdafdc-d707-42fa-ba74-98b30cf9ab2a - DT2...,US Market,Sarwa Trade
2023-02-17,buy,VOO,0.218967,372.07,-81.47,NaN,USD,Trade Entry,NYSEArca,Sarwa Trade
2023-02-17,buy,TSLA,0.202666,201.02,-40.74,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-02-17,buy,PERI,0.407270,33.34,-13.58,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-03-29,Net Dividend,VOO,NaN,NaN,0.24,NaN,USD,"Cash DIV @ 1.4874, Pos QTY: 0.218966527, Rec D...",NYSEArca,Sarwa Trade
...,...,...,...,...,...,...,...,...,...,...
2025-07-31,Net Dividend,GOF,NaN,NaN,0.39,NaN,USD,"Cash DIV @ 0.1821, Pos QTY: 3, Rec Date: 2025-...",NYSE,Sarwa Trade
2025-07-31,Net Dividend,BRW,NaN,NaN,0.28,NaN,USD,"Cash DIV @ 0.085, Pos QTY: 4.707717967, Rec Da...",NYSE,Sarwa Trade
2025-08-04,buy,VOO,0.426862,575.08,245.48,1.0,USD,Purchase of VOO shares on 2025-08-04,NYSEArca,Sarwa Trade - Interim


### Initialize and Assess Symbols

In [3]:
# 1. Initialize the Symbol manager
symbol_manager = Symbols(master_log)

# 2. Assess all symbols (checks cache and yfinance, creates templates for missing symbols)
symbol_manager.assess()

# 3. Display the results for user verification
print("✅ The following symbols were automatically identified:")
found_df = symbol_manager.get_found()
display(found_df)

print("❌ The following symbols were NOT found:")
missing_list = symbol_manager.get_missing()
print(missing_list)

✅ The following symbols were automatically identified:


,Name,Type,Exchange,Currency,Industry,Sector,Country,DataProvider
Symbol,,,,,,,,
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,Israel,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,United States,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance
...,...,...,...,...,...,...,...,...
JPMB,JPMorgan USD Emerging Markets Sovereign Bond ETF,etf,NYSEArca,USD,None,None,None,yfinance
SMOT,VanEck Morningstar SMID Moat ETF,etf,Cboe US,USD,None,None,None,yfinance
XLV,The Health Care Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance


❌ The following symbols were NOT found:
['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY']


### User Correction Step

Reviews the output from the cell above and lists any symbols that were incorrectly identified by the data provider (e.g. `yfinance`).

In [4]:
# User reviews the table above and populates this list
incorrectly_identified_symbols = ["AMR", "ASM"]

# Mark these symbols as user-provided, which also updates the metadata template
symbol_manager.mark_as_manual(incorrectly_identified_symbols)

Updating cache for incorrectly identified symbols: ['AMR', 'ASM']
Caches updated successfully.


### Manual Task

1) Open the file: `data/manual-source/metadata.json`.
2) Fill in the `null` values for all the symbols listed.
3) For each of those same symbols, add a price history CSV file to the `data/manual-source/` directory.

### Reload Metadata and Get Final DataFrame

In [5]:
# Reload the user-edited file from disk
symbol_manager.reload_user_metadata()

# Get the final, combined DataFrame of all symbol metadata
symbol_df = symbol_manager.get_unified_df()

print("--- Final Unified Symbol Metadata ---")
display(symbol_df)

Reloading user-provided metadata from disk...
User metadata reloaded successfully.
Successfully created unified symbols DataFrame.
--- Final Unified Symbol Metadata ---


,Name,Type,Exchange,Currency,Industry,Sector,Country,DataProvider
Symbol,,,,,,,,
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,Israel,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,United States,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance
...,...,...,...,...,...,...,...,...
IHC,International Holding Company PJSC,equity,ADX,AED,Industrials,Conglomerates,United Arab Emirates,manual
EMAAR,Emaar Properties PJSC,equity,DFM,AED,Real Estate - Development,Real Estate,United Arab Emirates,manual
MULTIPLY,Multiply Group PJSC,equity,ADX,AED,Financials,Asset Management,United Arab Emirates,manual


### Initialize and Run Portfolio Analysis

In [6]:
# Get the project date range
start_date, end_date, date_range, last_market_day = config.project_dates(
    master_log.index
)

# 1. Initialize the Portfolio analysis engine
portfolio = Portfolio(master_log, symbol_manager, date_range, last_market_day)

# 2. Run all calculations (fetching prices, calculating holdings and value)
portfolio.calculate_holdings_and_value()

portfolio_value = portfolio.get_total_value_history()

Portfolio object initialized.
Preparing trade log...
Fetching price and split data...
Successfully created unified symbols DataFrame.
Converting prices to base currency (USD)...
Successfully created unified symbols DataFrame.
Fetching FX rate for AEDUSD=X...
Calculating portfolio income...
Calculating daily holdings and value...
Calculating split-adjusted holdings and market value...
Calculating gains and returns with commissions...
Calculations complete.


### Display Portfolio Results

In [7]:
portfolio_value

Date
2023-02-16       0.000000
2023-02-17     134.849935
2023-02-18     134.849935
2023-02-19     134.849935
2023-02-20     134.849935
                 ...     
2025-08-03    3213.962371
2025-08-04    3506.821360
2025-08-05    3490.818051
2025-08-06    3511.104812
2025-08-07    3494.386119
Length: 904, dtype: float64

In [8]:
portfolio.get_current_holdings().round(12)

,Shares,Market Value (USD)
VOO,1.978926,1150.329627
VOOG,0.722995,297.122146
BTC-USD,0.000324,38.079207
XLK,1.223873,322.233632
SPY,0.177676,112.335697
MSFT,0.255834,133.248629
USRT,0.167590,9.448729
META,0.138582,105.576301
ADBE,0.000000,0.000000
NVDA,0.250397,45.264287


In [9]:
portfolio.get_return_summary()

,Income,Realized Gains,Unrealized Gains,Total Return
XLK,0.0,57.658297,270.484480,328.142777
VOO,0.0,11.159077,315.399939,326.559017
VOOG,0.0,59.000000,147.122146,206.122146
SPY,0.0,0.000000,112.335697,112.335697
META,0.0,0.000000,105.576301,105.576301
...,...,...,...,...
ARM,0.0,0.000000,-5.042104,-5.042104
BURJEEL,0.0,0.000000,-6.131706,-6.131706
TQQQ,0.0,-6.341035,0.000000,-6.341035
ROM,0.0,-8.084517,0.000000,-8.084517


In [10]:
portfolio.holdings["unrealized_gains"]

,VOO,TSLA,PERI,UAE,VOOG,BTC-USD,XLK,SPY,VHT,NOBL,...,ROM,ARM,RYAAY,APO,KKR,JPMB,SMOT,XLV,BRK-B,BX
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-16,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-17,79.078564,42.217437,13.553934,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-18,79.078564,42.217437,13.553934,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-19,79.078564,42.217437,13.553934,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-20,79.078564,42.217437,13.553934,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-03,298.476884,0.000000,0.000000,0.0,141.540621,7.515124,262.871986,110.464763,0.0,0.0,...,0.0,-4.879645,3.597288,-2.257825,-1.088029,-3.516160,-0.870683,-2.599214,-2.140553,-3.812024
2025-08-04,313.124247,0.000000,0.000000,0.0,146.652204,7.792059,269.260607,112.143804,0.0,0.0,...,0.0,-4.680004,4.117808,-0.263458,-0.112268,-3.220638,-0.525835,-2.055542,-3.842705,-3.443820
2025-08-05,307.602968,0.000000,0.000000,0.0,144.707344,7.490411,266.604785,111.575238,0.0,0.0,...,0.0,-4.907934,4.064178,1.618614,-1.046426,-3.222326,-0.604863,-2.203027,-3.212920,-4.362699


In [11]:
portfolio.holdings["realized_gains"]

,VOO,TSLA,PERI,UAE,VOOG,BTC-USD,XLK,SPY,VHT,NOBL,...,ROM,ARM,RYAAY,APO,KKR,JPMB,SMOT,XLV,BRK-B,BX
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-16,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-17,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-18,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-19,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-20,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-03,11.159077,75.986578,0.0,0.920065,59.0,0.010016,57.658297,0.0,89.604121,65.321491,...,-8.084517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-04,11.159077,75.986578,0.0,0.920065,59.0,0.010016,57.658297,0.0,89.604121,65.321491,...,-8.084517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-05,11.159077,75.986578,0.0,0.920065,59.0,0.010016,57.658297,0.0,89.604121,65.321491,...,-8.084517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
total_returns = (
    portfolio.holdings["unrealized_gains"]
    + portfolio.holdings["realized_gains"]
    + portfolio.holdings["income"]
)
total_returns

,VOO,TSLA,PERI,UAE,VOOG,BTC-USD,XLK,SPY,VHT,NOBL,...,ROM,ARM,RYAAY,APO,KKR,JPMB,SMOT,XLV,BRK-B,BX
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-17,79.078564,42.217437,13.553934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-18,79.078564,42.217437,13.553934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-19,79.078564,42.217437,13.553934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-20,79.078564,42.217437,13.553934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-03,309.635961,75.986578,0.000000,0.920065,200.540621,7.525140,320.530284,110.464763,89.604121,65.321491,...,-8.084517,-4.879645,3.597288,-2.257825,-1.088029,-3.516160,-0.870683,-2.599214,-2.140553,-3.812024
2025-08-04,324.283325,75.986578,0.000000,0.920065,205.652203,7.802075,326.918904,112.143804,89.604121,65.321491,...,-8.084517,-4.680004,4.117808,-0.263458,-0.112268,-3.220638,-0.525835,-2.055542,-3.842705,-3.443820
2025-08-05,318.762045,75.986578,0.000000,0.920065,203.707344,7.500427,324.263082,111.575238,89.604121,65.321491,...,-8.084517,-4.907934,4.064178,1.618614,-1.046426,-3.222326,-0.604863,-2.203027,-3.212920,-4.362699


Ensure commissions are factored in. (Current amount excludes commission fee)

In [13]:
fig_holdings = go.Figure()

for symbol in total_returns.columns:
    fig_holdings.add_trace(
        go.Scatter(
            x=total_returns.index,
            y=total_returns[symbol],
            mode="lines",
            name=symbol,
            line=dict(width=1),
        )
    )

fig_holdings.update_layout(
    title_text="<b>Individual Holding Performance Over Time</b>",
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title=f"Total Returns ({config.BASE_CURRENCY})",
)

fig_holdings.show()

### Initialize and Run the Benchmark Simulation

In [14]:
# Create an instance of the Benchmark class
benchmark_simulation = Benchmark(master_log, date_range, last_market_day)

# Run benchmark simulation
benchmark_simulation.run_simulation()

Benchmark object initialized.
Fetching market data for benchmark: VOO...
Preparing and converting cash flows to base currency (USD)...
Fetching FX rate for AEDUSD=X...
Running benchmark simulation...
Benchmark simulation complete.


### Display Benchmark Result

In [15]:
# Now, you can get the populated DataFrame
benchmark_results = benchmark_simulation.get_results()

# Display the tail of the results to verify it worked
benchmark_results

,Open,Close,Dividends,Market,NetDeposit,Shares,DividendCash,TradeCash,Commission,NetDividend,PortfolioValue,TotalValue,TradeTrigger
Date,,,,,,,,,,,,,
2023-02-16,362.534207,362.080627,0.0,Open,271.57,0.000000,0.0,271.57,0.0,0.0,0.000000,0.000000,Buy
2023-02-17,360.140861,361.144531,0.0,Open,0.00,0.751289,0.0,0.00,1.0,0.0,271.324047,271.324047,None
2023-02-18,360.140861,361.144531,0.0,Closed,0.00,0.751289,0.0,0.00,0.0,0.0,271.324047,271.324047,None
2023-02-19,360.140861,361.144531,0.0,Closed,0.00,0.751289,0.0,0.00,0.0,0.0,271.324047,271.324047,None
2023-02-20,360.140861,361.144531,0.0,Closed,0.00,0.751289,0.0,0.00,0.0,0.0,271.324047,271.324047,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-03,575.729980,571.450012,0.0,Closed,0.00,6.282150,0.0,0.00,0.0,0.0,3589.934691,3589.934691,None
2025-08-04,575.169983,580.140015,0.0,Open,244.41,6.705347,0.0,0.00,1.0,0.0,3890.039890,3890.039890,None
2025-08-05,580.789978,577.349976,0.0,Open,0.00,6.705347,0.0,0.00,0.0,0.0,3871.331711,3871.331711,None


### Visualize Portfolio & Benchmark Performance

In [16]:
# --- 1. Prepare data for the chart ---
# Resample the daily net deposits to get the total for each month
monthly_deposits = benchmark_results["NetDeposit"].resample("ME").sum()

# --- 2. Create the Plotly Figure ---
fig = make_subplots(specs=[[{"secondary_y": True}]])

# --- 3. Add Traces to the Figure ---

# Trace 1: Personal Portfolio Value
fig.add_trace(
    go.Scatter(
        x=portfolio_value.index,
        y=portfolio_value,
        mode="lines",
        name="Personal Portfolio",
        line=dict(color="green", width=2),
    ),
    secondary_y=False,
)

# Trace 2: Benchmark Value
fig.add_trace(
    go.Scatter(
        x=benchmark_results.index,
        y=benchmark_results["TotalValue"],
        mode="lines",
        name=f"{config.BENCHMARK_INDEX} Benchmark",
        line=dict(color="red", width=2),
    ),
    secondary_y=False,
)

# Trace 3: Cumulative Net Deposits
fig.add_trace(
    go.Scatter(
        x=benchmark_results.index,
        y=benchmark_results["NetDeposit"].cumsum(),
        mode="lines",
        name="Cumulative Net Deposits",
        line=dict(color="darkgrey", width=1, dash="dash"),
    ),
    secondary_y=False,
)

# Trace 4: Monthly Deposits/Withdrawals (on secondary y-axis)
fig.add_trace(
    go.Bar(
        x=monthly_deposits.index,
        y=monthly_deposits,
        name="Deposits / Withdrawals",
        marker_color="royalblue",
        opacity=0.4,
    ),
    secondary_y=True,
)

# --- 5. Update Layout and Styling ---
fig.update_layout(
    title_text=f"Portfolio Performance vs. {config.BENCHMARK_INDEX} Benchmark",
    template="plotly_white",
    barmode="relative",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis=dict(
        title="Date",
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
)

# Update Y-Axes Titles
fig.update_yaxes(
    title_text=f"<b>Portfolio Value ({config.BASE_CURRENCY})</b>", secondary_y=False
)
fig.update_yaxes(
    title_text="<b>Monthly Cash Flow</b>",
    secondary_y=True,
    showgrid=False,
    layer="below traces",
)

fig.show()

In [17]:
portfolio_income = portfolio.get_monthly_income()
benchmark_income = benchmark_simulation.get_monthly_income()

fig_income_comp = go.Figure()

fig_income_comp.add_trace(
    go.Bar(
        x=portfolio_income.index,
        y=portfolio_income,
        name="Portfolio Income",
        marker_color="mediumseagreen",
    )
)

fig_income_comp.add_trace(
    go.Bar(
        x=benchmark_income.index,
        y=benchmark_income,
        name=f"{config.BENCHMARK_INDEX} Benchmark Income",
        marker_color="grey",
    )
)

fig_income_comp.update_layout(
    title_text="<b>Monthly Income Comparison</b>",
    template="plotly_white",
    barmode="group",
    xaxis_title="Date",
    yaxis_title=f"Net Income ({config.BASE_CURRENCY})",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig_income_comp.show()